# Introduction:

In these series of notebook we expanded the work with our feature space. We reduced smartly the number of features. We  also analyzed variables with regard the predictor we want to classify. Some ideas to ponder and you can think about what we can do with our variables and what are the effect of this actions:

+ Remove features with little variance. (imagine an extreme a zero variance feature, don't teach us anything!)
+ Univariate feauture selection. (There are a lot here, but I want to explore mutual information and corrolation.)
+ Recursive Feature elimination. (I will not use this one, I prefer creating a simple model and see different variables.)
+ Model with L1 regularization. (this model if it finds a variable with little predicting power it will just make it zero.)

My first ever repo have more about EDA and explaining what is behind the data. But this repo my goal is to create useful analysis that can be easy to replicate on your notebook, and taking a whole bunch of lines of code + text make it harder to look where the nice rounded functions are exactly.

In [125]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
# new library, we haven't used this one before in this repo
from sklearn.preprocessing import TargetEncoder

# remove feature with zero variance by default
from sklearn.feature_selection import VarianceThreshold

from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import r_regression

In [106]:
# Read the data
# This data you can find here: https://www.kaggle.com/c/home-data-for-ml-course/data

X_full = pd.read_csv('train.csv', index_col='Id')

# SalePrice is the target, if there is no target eliminate row associated with it
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X = X_full.copy()
X.drop(['SalePrice'], axis=1, inplace=True)

# Univariate feature selection

In [107]:
# It is always good idea to check the input of a pipeline that only transform and its output!
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [108]:
def simple_pipeline(X,y):

    columns = X.columns
    numerical_col = [col for col in X.columns if str(X[col].dtypes)!='object' ]
    categorical_col = [col for col in X.columns if str(X[col].dtypes)=='object' ]
    
   
    numerical_transformer = Pipeline(steps=[ 
            ("zero_variance_del",VarianceThreshold()),
            ("imputer", KNNImputer(n_neighbors=3))
          ])
    target_transformer =  Pipeline(steps=[
        ('encoder', TargetEncoder(smooth="auto",target_type='continuous')),
        ("zero_variance_del",VarianceThreshold())
    ])

    preprocessor = ColumnTransformer(transformers=
        [("numerical_transformer", numerical_transformer, numerical_col),
         ("target_transformer", target_transformer, categorical_col)
        ])
    
    # Bundle preprocessing and modeling code in a pipeline
    pipe = Pipeline(steps=[('preprocessor', preprocessor)
                         ])
    new_X = pipe.fit_transform(X,y)
    df = pd.DataFrame(new_X, columns=numerical_col+categorical_col)
    return df, pipe

In [109]:
X, pipe = simple_pipeline(X,y)

In [114]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,204583.434065,205193.355981,188693.648570,189131.470692,187980.035762,181633.013888,188870.432471,183243.332961,174598.898335,176696.021425
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,202677.690014,198773.207817,186713.078652,187234.691117,186190.143607,179744.610076,186506.696623,181362.480866,172711.865799,174090.959192
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,201873.710170,201307.241896,187034.222611,186888.409831,185285.951516,179490.830467,187613.807755,180669.768103,172660.545648,174740.142821
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,134180.402274,141366.045480,188693.648570,189131.470692,187980.035762,181633.013888,188870.432471,183243.332961,174598.898335,142379.141825
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,202726.354206,200528.189375,188084.201591,188426.015726,186678.312141,180631.904139,187472.310831,182554.156452,173718.356214,175658.370808


In [121]:
(X.dtypes!="float64")

MSSubClass       False
LotFrontage      False
LotArea          False
OverallQual      False
OverallCond      False
                 ...  
PoolQC           False
Fence            False
MiscFeature      False
SaleType         False
SaleCondition    False
Length: 79, dtype: bool

In [131]:
pd.set_option('display.max_rows', None)

In [132]:
def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y, discrete_features='auto')
    r_score = r_regression(X, y)
    # we could also use f_regression to see if the corrolation is significant or not
    # but in this case I just wanted to see amount how those variables move together.
    return pd.DataFrame({"mi_scores":mi_scores,
                        "r_score": r_score}, index=X.columns).sort_values(by='r_score',ascending=False)

mi_scores = make_mi_scores(X, y)
mi_scores  # show a few features with their MI scores

,mi_scores,r_score
OverallQual,0.574649,0.790982
Neighborhood,0.487468,0.729628
GrLivArea,0.481396,0.708624
ExterQual,0.323506,0.688778
BsmtQual,0.312515,0.678865
KitchenQual,0.304606,0.673661
GarageCars,0.355246,0.640409
GarageArea,0.365603,0.623431
TotalBsmtSF,0.365849,0.613581
1stFlrSF,0.310158,0.605852
